<b>ССЫЛКА НА ДАШБОРД:</b>
(в процессе решения)

- Выручка, полученная в этот день.
- Суммарная выручка на текущий день.
- Прирост выручки, полученной в этот день, относительно значения выручки за предыдущий день.

In [ ]:
with date_revenue as(
select date,
       round(sum(price)) as revenue
from products
join (select creation_time::date as date,
              unnest(product_ids) as product_id
      from orders
      where order_id not in (select order_id
                             from user_actions
                             where action = 'cancel_order')) as t1 using(product_id)
group by date
order by date
),

date_total_revenue as(
select date,
       revenue,
       sum(revenue) over(order by date) as total_revenue
from date_revenue
)

select date,
       revenue,
       total_revenue,
       round(revenue :: decimal / lag(revenue, 1) over(ORDER BY date) * 100 - 100, 2) as revenue_change
from date_total_revenue


- Выручка на пользователя (ARPU) за текущий день.
- Выручка на платящего пользователя (ARPPU) за текущий день.
- Выручка с заказа, или средний чек (AOV) за текущий день.

In [ ]:
with total_revenue as(
select date,
       sum(price) as sum_price
from products
join (select creation_time::date as date,
             order_id,
             unnest(product_ids) as product_id
      from orders
      where order_id not in(select order_id from user_actions where action = 'cancel_order')) as t1 using(product_id)
group by date
),

total_users as (
select time::date as date,
       count(distinct user_id) as user_count
from user_actions
group by date
),

paying_users as (
select time::date as date,
       count(distinct user_id) as paying_user_count
from user_actions
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
),

total_orders as (
select creation_time::date as date,
       count(order_id) as order_count
from orders
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
)


select date,
       round(sum_price::decimal / user_count, 2) as arpu,
       round(sum_price::decimal / paying_user_count, 2) as arppu,
       round(sum_price::decimal / order_count, 2) as aov
from total_revenue
join total_users using(date)
join paying_users using(date)
join total_orders using(date)
order by date


- Накопленная выручка на пользователя (Running ARPU).
- Накопленная выручка на платящего пользователя (Running ARPPU).
- Накопленная выручка с заказа, или средний чек (Running AOV).

In [ ]:
with total_revenue as (SELECT date,
                              sum(price) as sum_price
                       FROM  products
                       join (SELECT creation_time::date as date,
                                                    order_id,
                                                    unnest(product_ids) as product_id
                                             FROM orders
                                             WHERE order_id not in (SELECT order_id
                                                                     FROM   user_actions
                                                                     WHERE  action = 'cancel_order')) as t1 using (product_id)
                                            GROUP BY date),

new_users as (SELECT time::date as date,
                    count(distinct user_id) as new_users
              FROM   user_actions
              WHERE  (time::date, user_id) in (SELECT min(time::date),
                                                      user_id
                                               FROM   user_actions
                                               GROUP BY user_id)
              GROUP BY date),
total_users as (SELECT date,
                       sum(new_users) OVER (ORDER BY date) as total_user_count
                FROM  new_users),
paying_users as (SELECT time::date as date,
                        count(distinct user_id) as paying_user_count
                FROM user_actions
                WHERE order_id not in (SELECT order_id
                                              FROM user_actions
                                       WHERE  action = 'cancel_order')
                                       and (user_id, time::date) in (SELECT user_id,
                                                                         min(time::date)
                                                                  FROM  user_actions
                                                                  WHERE  order_id not in (SELECT order_id
                                                                                          FROM   user_actions
                                                                                          WHERE  action = 'cancel_order')
                GROUP BY date),
total_orders as (SELECT creation_time::date as date,
                        count(order_id) as order_count
                FROM  orders
                WHERE order_id not in (SELECT order_id
                                      FROM  user_actions
                                      WHERE  action = 'cancel_order')
                GROUP BY date)
SELECT date,
       round(sum(sum_price) OVER (ORDER BY date) / total_user_count, 2) as running_arpu,
       round(sum(sum_price) OVER (ORDER BY date) / sum(paying_user_count) OVER(ORDER BY date), 2) as running_arppu,
       round(sum(sum_price) OVER (ORDER BY date) / sum(order_count) OVER (ORDER BY date), 2) as running_aov
FROM   total_revenue
join total_users using (date)
join paying_users using (date)
join total_orders using (date)
ORDER BY date;